In [ ]:
%pylab inline

In [ ]:
# Keras
import tensorflow as tf
from tensorflow.contrib import keras
from tensorflow.contrib.keras.python.keras import models, losses
from tensorflow.contrib.keras.python.keras.layers.core import Dense,Reshape,Flatten, Dropout
from tensorflow.contrib.keras.python.keras.layers.pooling import MaxPool2D
from tensorflow.contrib.keras.python.keras.layers.convolutional import ZeroPadding2D, Conv2D
from tensorflow.contrib.keras.python.keras.layers.merge import Concatenate, Add
from tensorflow.contrib.keras.python.keras.layers.recurrent import LSTM, GRU
from tensorflow.contrib.keras.python.keras.layers.pooling import MaxPool2D
from tensorflow.contrib.keras.python.keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
# basic
import os
import random
import json
import pandas as pd
import seaborn as sns
sns.set(style="darkgrid")

# geo-related
from osgeo import gdal
import geopandas as gpd
import folium
from folium import plugins

# scikit-learn
from sklearn import model_selection
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc

from rscnn_utils import utils

In [ ]:
seed = 7
random.seed(seed)
np.random.seed(seed)
tf.set_random_seed(seed)

In [ ]:
folder = './buildData/'

ht = gdal.Open(os.path.join(folder,'mosaic_post.tif'))
gt = ht.GetGeoTransform()

buildings_df = gpd.read_file(os.path.join(folder,'building_post_id_join_reclassified.shp'))
buildings_df['image_id']='mosaic_post'

In [ ]:
width = ht.RasterXSize    
height = ht.RasterYSize
minx = gt[0]   
miny = gt[3] + width * gt[4] + height * gt[5]    
maxx = gt[0] + width * gt[1] + height * gt[2]    
maxy = gt[3]

In [ ]:
map_osm = folium.Map([18.54,-72.34],
                  zoom_start=14,
                  tiles='cartodbpositron')

band1 = ht.GetRasterBand(1).ReadAsArray()

# https://openlayers.org/en/latest/examples/static-image.html
plugins.ImageOverlay(band1[0::20,0::20],
                     [[miny, minx], [maxy, maxx]],
                     opacity=0.8).add_to(map_osm)

gjson = buildings_df.geometry.to_json()
poly = folium.features.GeoJson(gjson).add_to(map_osm)
map_osm

In [ ]:
buildings_df['raw_index'] = buildings_df.index
print(buildings_df.crs)
buildings_df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(3,5))

buildings_df['Destruction'] = buildings_df.apply(lambda x: 'Destruction' if x['Damage_ID'] =='GRADE 5 Destruction' else 'Non-destrctuion', axis=1)

In [ ]:
utils = utils()

In [ ]:
chips_dim = buildings_df.apply(lambda poly: pd.Series(utils.chip_dim(poly['geometry'],poly['image_id'])),axis=1)

In [ ]:
buildings_df = pd.concat([buildings_df, chips_dim], axis=1, join_axes=[chips_dim.index])
buildings_df.shape

In [ ]:
min_dim = 10
max_dim = 100

chips_dim_min = chips_dim.apply(lambda chip: chip[0]>min_dim and chip[1]>min_dim,axis=1)
chips_dim_max = chips_dim.apply(lambda chip: chip[0]<max_dim and chip[1]<max_dim,axis=1)

chips_filter_dim = chips_dim_min & chips_dim_max
print (chips_filter_dim.describe())

In [ ]:
buildings_filter = buildings_df[chips_filter_dim]
buildings_filter.reset_index(drop=True,inplace=True)

In [ ]:
chip_uni_ = []
for x in range(len(buildings_filter)):
    chip = utils.vector_clip_raster(buildings_filter.loc[x,'geometry'],buildings_filter.loc[x,'image_id'])         
    chip = np.transpose(chip,(1,2,0))   ################## transpose ########
    chip = utils.uniform_chips(chip,resize=False,max_dim=max_dim)
    chip_uni_.append(chip)
chip_uni = np.array(chip_uni_)
chip_uni.nbytes/10**6

In [ ]:
for i in range(8):
    plt.subplot(2, 4, i+1)
    image = chip_uni[i, :, :, :]
    plt.imshow(image)#, cmap='gray'
    plt.axis('off')
plt.tight_layout()

### Modelling

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(
                    chip_uni, buildings_filter['Destruction'].values, test_size=0.33, random_state=42)

In [ ]:
positive = 0
negative = 1
# one-hot-label
y_train_digits = [negative if bild=='Destruction' else positive for bild in y_train]
y_train_ohl = keras.utils.to_categorical(y_train_digits, num_classes=2)

y_true = [negative if bild=='Destruction' else positive for bild in y_test]

print(y_train[0])
print(y_train_digits[0])
print(y_train_ohl[0])

In [ ]:
from squeezenet import smallSqueezeNet

In [ ]:
input_shape = (max_dim,max_dim,3)

def sqzmodel(optimizer='adam', init='he_uniform',summary=False):
    sqzNet = smallSqueezeNet(input_shape=input_shape, init=init)
    sqzNet.compile(optimizer=optimizer, loss = losses.categorical_crossentropy, metrics=['accuracy'])
    if summary:
        sqzNet.summary()
    return sqzNet

In [ ]:
kModel = KerasClassifier(build_fn=sqzmodel, batch_size=128, epochs=50, verbose=0)
hist = kModel.fit(x=X_train, y=y_train_ohl,validation_split=0.1)

In [ ]:
y_pred = kModel.model.predict(X_test)
y_pred = np.argmax(y_pred,axis=1)

In [ ]:
cm = confusion_matrix(y_true, y_pred)
print (cm)

print(accuracy_score(y_true, y_pred))
print (precision_score(y_true,y_pred))
print (recall_score(y_true,y_pred))
print (f1_score(y_true,y_pred))

In [ ]:
print(classification_report(y_true, y_pred, target_names=['Non-Destruction','Destruction']))

In [ ]:
y_pred_prob = kModel.model.predict(X_test)
plot_roc_curve(y_true, y_pred_prob[:,1])

### Under Sampling

In [ ]:
from imblearn.under_sampling import OneSidedSelection

In [ ]:
pca = PCA(n_components=30)
X_train_PCA = pca.fit_transform(X_train.reshape(X_train.shape[0],-1))
X_train_PCA.shape

In [ ]:
oss = OneSidedSelection(return_indices=True,random_state=42)
X_resampled, y_resampled, idx_resampled = oss.fit_sample(X_train_PCA, y_train)

In [ ]:
np.random.shuffle(idx_resampled)

X_train_us = X_train[idx_resampled]
y_train_us = y_train[idx_resampled]

X_train_us.shape,y_train_us.shape

In [ ]:
y_train_us_digits = [negative if bild=='Destruction' else positive for bild in y_train_us]
y_train_us_ohl = keras.utils.to_categorical(y_train_us_digits, num_classes=2)

In [ ]:
kModel = KerasClassifier(build_fn=sqzmodel, batch_size=128, epochs=50, verbose=0)
hist = kModel.fit(x=X_train_us, y=y_train_us_ohl,validation_split=0.1)

In [ ]:
y_pred=kModel.model.predict(X_test)
y_pred = np.argmax(y_pred,axis=1)

In [ ]:
cm = confusion_matrix(y_true, y_pred)
print (cm)

In [ ]:
print('\n')
print(accuracy_score(y_true, y_pred))
print (precision_score(y_true,y_pred))
print (recall_score(y_true,y_pred))
print (f1_score(y_true,y_pred))

In [ ]:
print(classification_report(y_true, y_pred, target_names=['Non-Destruction','Destruction']))

In [ ]:
y_pred_prob = kModel.model.predict_on_batch(X_test)
plot_roc_curve(y_true, y_pred_prob[:,1])

### Comparison

In [ ]:
# import h5py

# f = h5py.File('./models/'+base_model_name+'.h5', 'r+')
# del f['optimizer_weights']
# f.close()

In [ ]:
sns.set(font_scale=1.2)

from pylab import rcParams
rcParams['figure.figsize'] = 5, 5

In [ ]:
demo_index = [74,700,703,64]
demos = X_test[demo_index, :, :, :]

for i in np.arange(shape(demos)[0]):
    plt.subplot(1, 4, i+1)
    img = demos[i,:,:,:]
    plt.imshow(img)
    plt.axis('off')
plt.tight_layout()

In [ ]:
demo_pred = base_model.predict(demos)
np.around(demo_pred,3)

In [ ]:
demo_pred = oss_model.predict(demos)
np.around(demo_pred,3)

In [ ]:
_, Index_test= model_selection.train_test_split(buildings_filter['raw_index'].values, test_size=0.33, random_state=42)

In [ ]:
def reclassify(x):
    if x[0]==0 and x[1]==0:
        return 'TN'
    elif x[0]==0 and x[1]==1:
        return 'FN'
    elif x[0]==1 and x[1]==1:
        return 'TD'
    elif x[0]==1 and x[1]==0:
        return 'FD'
    else:
        return 0

In [ ]:
base_tr_pr = gpd.GeoDataFrame([list(y_true), list(base_y_pred)]).T #{'init':'epsg:4326'}
base_tr_pr.columns = ['label','sqz']

base_tr_pr['classify'] = base_tr_pr.apply(reclassify, axis=1)
base_tr_pr.index = Index_test
base_tr_pr['geometry'] = buildings_df.loc[Index_test,'geometry']
base_tr_pr.crs={'init':'epsg:4326'}

base_tr_pr.to_file('./base_model.shp', driver = 'ESRI Shapefile')

In [ ]:
oss_tr_pr = gpd.GeoDataFrame([list(y_true), list(oss_y_pred)]).T #{'init':'epsg:4326'}
oss_tr_pr.columns = ['label','oss_sqz']

oss_tr_pr['classify'] = oss_tr_pr.apply(reclassify, axis=1)
oss_tr_pr.index = Index_test
oss_tr_pr['geometry'] = buildings_df.loc[Index_test,'geometry']
oss_tr_pr.crs={'init':'epsg:4326'}

oss_tr_pr.to_file('./oss_model.shp', driver = 'ESRI Shapefile')